In [314]:
# movielens 100k
import pandas as pd
rating=pd.read_csv('dataset/ml-100k/u.data',sep='\t',names=['user_id','movie_id','rating','timestamp'])
rating=rating.drop('timestamp',axis=1)


In [315]:
#userinfo
userinfo=pd.read_csv('dataset/ml-100k/u.user',sep='|', names=['age','gender','occupation','zipcode'])


In [316]:
# need to check movie frequency and user frequency
# movieinfo
rating['movie_id'].value_counts()
# show movies where value count is more than 10
rating['movie_id'].value_counts()[rating['movie_id'].value_counts()>10]

# for every rating, i want to add column named 'movie_frequency' and 'user_frequency'

# add movie_frequency
rating['movie_frequency']=rating['movie_id'].map(rating['movie_id'].value_counts())
# add user_frequency
rating['user_frequency']=rating['user_id'].map(rating['user_id'].value_counts())


In [317]:
rating

,user_id,movie_id,rating,movie_frequency,user_frequency
0,196,242,3,117,39
1,186,302,3,297,92
2,22,377,1,13,128
3,244,51,2,81,238
4,166,346,1,126,20
...,...,...,...,...,...
99995,880,476,3,160,368
99996,716,204,5,350,269
99997,276,1090,1,37,518
99998,13,225,2,109,636


In [318]:
import copy
df=copy.deepcopy(rating)

In [319]:
df

,user_id,movie_id,rating,movie_frequency,user_frequency
0,196,242,3,117,39
1,186,302,3,297,92
2,22,377,1,13,128
3,244,51,2,81,238
4,166,346,1,126,20
...,...,...,...,...,...
99995,880,476,3,160,368
99996,716,204,5,350,269
99997,276,1090,1,37,518
99998,13,225,2,109,636


In [320]:
from copy import deepcopy
# function taht calculates the c value for each customer and product
# the higher beta means more weiht on the product frequency, the higher alpha means more weight on the customer frequency
def get_c(df, alpha=.5, beta=.5, gamma=.5, c_zero=.5):
    UF = np.array(df["user_frequency"].astype("float"), dtype=float)
    UF /= np.sum(UF)
    IF = np.array(df["movie_frequency"].astype("float"), dtype=float)
    IF /= np.sum(IF)
    Fs = alpha * beta * IF * UF
    Fs_gamma = Fs ** gamma
    c = Fs_gamma / np.sum(Fs_gamma)
    c = c_zero * c / np.max(c)
    c_appended_df = deepcopy(df)
    c_appended_df['c'] = c

    return c_appended_df

unique_customers = rating['user_id'].unique()
import numpy as np
import tqdm
not_purchased_df = pd.DataFrame()
ns_df_list = []
df['user_frequency'] = df.groupby('user_id')['user_id'].transform('count')
df['movie_frequency'] = df.groupby('movie_id')['movie_id'].transform('count')
#multiprocess
for customer in tqdm.tqdm(unique_customers):
    unique_products = df['movie_id'].unique()


    customer_frequency = df[df['user_id'] == customer]['user_frequency'].iloc[0]
    purchased_products = df[df['user_id'] == customer]['movie_id'].unique()

    #customer_birth_category = df[df['user_id'] == customer]['BIRTH_YEAR'].iloc[0]
    #customer_gender_category = df[df['user_id'] == customer]['GENDER'].iloc[0]

    not_purchased_df_all = df[~df['movie_id'].isin(purchased_products)]
    not_purchased_codes = not_purchased_df_all['movie_id'].unique()
    negative_sample_products=np.random.choice(not_purchased_codes, int(len(not_purchased_codes) *0.1),replace=False)
    ns_test=df[df['movie_id'].isin(negative_sample_products)][['movie_id','movie_frequency']]
    ns_test=ns_test.drop_duplicates(subset=['movie_id'],keep='first',inplace=False)
    ns_df=pd.DataFrame()

    ns_df['movie_id'] = negative_sample_products
    ns_df=ns_df.assign(user_id=customer)
    #ns_df['AUTH_CUSTOMER_ID'] = customer
    #ns_df=ns_df.assign(BIRTH_YEAR = customer_birth_category)
    #ns_df=ns_df.assign(GENDER= customer_gender_category)
    ns_df=ns_df.assign(user_frequency = customer_frequency)
    ns_df=ns_df.join(ns_test.set_index('movie_id'), on='movie_id')
    # not_purchased_df=pd.concat([not_purchased_df,ns_df],axis=0, ignore_index=True)
    ns_df_list += [ns_df]
not_purchased_df=pd.concat(objs=ns_df_list, axis=0, ignore_index=True)
# not_purchased_df["GENDER"] = not_purchased_df["GENDER"].replace("MALE", 0).replace("FEMALE", 1).astype("float")
# not_purchased_df = get_c(not_purchased_df, 0.5, 0.5, 0.5, 0.5) # please adjest weight
# not_purchased_df['target'] = 0

100%|██████████| 943/943 [00:09<00:00, 101.49it/s]


In [321]:
not_purchased_df=get_c(not_purchased_df,0.5,0.5,0.5,0.5)

In [322]:
not_purchased_df['target']=0

In [323]:
not_purchased_df

,movie_id,user_id,user_frequency,movie_frequency,c,target
0,416,196,39,64,0.044425,0
1,1582,196,39,1,0.005553,0
2,1043,196,39,8,0.015707,0
3,841,196,39,48,0.038473,0
4,785,196,39,39,0.034679,0
...,...,...,...,...,...,...
148176,378,941,22,101,0.041916,0
148177,1078,941,22,22,0.019563,0
148178,823,941,22,82,0.037768,0
148179,539,941,22,44,0.027666,0


In [324]:
rating['c']=1
rating['target']=1

In [325]:
rating.drop('rating',axis=1,inplace=True)

In [326]:
rating

,user_id,movie_id,movie_frequency,user_frequency,c,target
0,196,242,117,39,1,1
1,186,302,297,92,1,1
2,22,377,13,128,1,1
3,244,51,81,238,1,1
4,166,346,126,20,1,1
...,...,...,...,...,...,...
99995,880,476,160,368,1,1
99996,716,204,350,269,1,1
99997,276,1090,37,518,1,1
99998,13,225,109,636,1,1


In [327]:
# want to merge two dataframe rating and not_purchased_df but their columns sequence is different so we need to change the sequence
not_purchased_df=not_purchased_df[['user_id','movie_id','user_frequency','movie_frequency','c','target']]

In [328]:
rating

,user_id,movie_id,movie_frequency,user_frequency,c,target
0,196,242,117,39,1,1
1,186,302,297,92,1,1
2,22,377,13,128,1,1
3,244,51,81,238,1,1
4,166,346,126,20,1,1
...,...,...,...,...,...,...
99995,880,476,160,368,1,1
99996,716,204,350,269,1,1
99997,276,1090,37,518,1,1
99998,13,225,109,636,1,1


In [329]:
not_purchased_df

,user_id,movie_id,user_frequency,movie_frequency,c,target
0,196,416,39,64,0.044425,0
1,196,1582,39,1,0.005553,0
2,196,1043,39,8,0.015707,0
3,196,841,39,48,0.038473,0
4,196,785,39,39,0.034679,0
...,...,...,...,...,...,...
148176,941,378,22,101,0.041916,0
148177,941,1078,22,22,0.019563,0
148178,941,823,22,82,0.037768,0
148179,941,539,22,44,0.027666,0


In [330]:
rating=pd.concat([rating,not_purchased_df],axis=0,ignore_index=True)

In [331]:
rating

,user_id,movie_id,movie_frequency,user_frequency,c,target
0,196,242,117,39,1.000000,1
1,186,302,297,92,1.000000,1
2,22,377,13,128,1.000000,1
3,244,51,81,238,1.000000,1
4,166,346,126,20,1.000000,1
...,...,...,...,...,...,...
248176,941,378,101,22,0.041916,0
248177,941,1078,22,22,0.019563,0
248178,941,823,82,22,0.037768,0
248179,941,539,44,22,0.027666,0


In [332]:

# drop zip code and one hot encode gender and occupation in userinfo daframe
userinfo=userinfo.drop('zipcode',axis=1)

userinfo=pd.get_dummies(columns=['occupation'],data=userinfo)

In [333]:
userinfo

,age,gender,occupation_administrator,occupation_artist,occupation_doctor,occupation_educator,occupation_engineer,occupation_entertainment,occupation_executive,occupation_healthcare,...,occupation_marketing,occupation_none,occupation_other,occupation_programmer,occupation_retired,occupation_salesman,occupation_scientist,occupation_student,occupation_technician,occupation_writer
1,24,M,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,53,F,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
3,23,M,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,24,M,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
5,33,F,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,26,F,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
940,32,M,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
941,20,M,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
942,48,F,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [334]:
import numpy as np

In [335]:
userinfo 
# make user_id column as index
#userinfo=userinfo.set_index(np.array(sorted(rating['user_id'].unique())))
userinfo['user_id']=userinfo.index

In [336]:
# if gender is M then 1 else 0 in userinfo dataframe
temp=[]
for i in userinfo['gender']:
    if i=='M':
        i=1
    else:
        i=0
    temp.append(i)
    

In [337]:
userinfo['gender'] =temp


In [338]:
movieinfo=pd.read_csv('dataset/ml-100k/u.item',sep='|',names=['movie_id','movie_title','release_date','video_release_date','IMDb_URL','unknown','Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama','Fantasy','Film_Noir','Horror','Musical','Mystery','Romance','Sci_Fi','Thriller','War','Western'],encoding='latin-1')

In [339]:
movieinfo.drop(['movie_title','release_date','video_release_date','IMDb_URL'],axis=1,inplace=True)

In [340]:
rating

,user_id,movie_id,movie_frequency,user_frequency,c,target
0,196,242,117,39,1.000000,1
1,186,302,297,92,1.000000,1
2,22,377,13,128,1.000000,1
3,244,51,81,238,1.000000,1
4,166,346,126,20,1.000000,1
...,...,...,...,...,...,...
248176,941,378,101,22,0.041916,0
248177,941,1078,22,22,0.019563,0
248178,941,823,82,22,0.037768,0
248179,941,539,44,22,0.027666,0


In [341]:
movieinfoadded=pd.merge(rating,movieinfo,on='movie_id',how='left')

In [342]:
userinfoadded=pd.merge(movieinfoadded,userinfo,on='user_id',how='left')

In [343]:
data_one_hot=pd.get_dummies(columns=['user_id','movie_id'],data=userinfoadded)

In [344]:
data_one_hot

,movie_frequency,user_frequency,c,target,unknown,Action,Adventure,Animation,Children,Comedy,...,movie_id_1673,movie_id_1674,movie_id_1675,movie_id_1676,movie_id_1677,movie_id_1678,movie_id_1679,movie_id_1680,movie_id_1681,movie_id_1682
0,117,39,1.000000,1,0,0,0,0,0,1,...,False,False,False,False,False,False,False,False,False,False
1,297,92,1.000000,1,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
2,13,128,1.000000,1,0,0,0,0,1,1,...,False,False,False,False,False,False,False,False,False,False
3,81,238,1.000000,1,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
4,126,20,1.000000,1,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248176,101,22,0.041916,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
248177,22,22,0.019563,0,0,0,0,1,1,0,...,False,False,False,False,False,False,False,False,False,False
248178,82,22,0.037768,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
248179,44,22,0.027666,0,0,0,0,0,1,1,...,False,False,False,False,False,False,False,False,False,False


In [345]:
data_one_hot.to_pickle('dataset/ml-100k/data_one_hot.pkl')